In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_path = "./input/train_local.csv"
test_path = "./input/test_local.csv"

In [3]:
import hashlib

In [4]:
s="A"
int(hashlib.sha256(s.encode('utf-8')).hexdigest(), 16) % 10**2

45

In [5]:
def hash_cat(s):
    return int(hashlib.sha256(s.encode('utf-8')).hexdigest(), 16) % 10**3

In [6]:
# x = ['A', "b"]


In [7]:
# x.index('c')

In [8]:
cat_dict = []
def hash_cat_two(s):
    if s not in cat_dict:
        cat_dict.append(s)
    return cat_dict.index(s)

In [9]:
import pandas as pd

In [10]:
df_train = pd.read_csv(train_path)

In [11]:
len(df_train)

240000

In [12]:
df_train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,495701,A,J,A,A,F,O,C,E,AX,...,0.387487,0.251159,0.801758,0.553776,0.444780,0.335810,0.345248,0.470157,0.319817,0
1,475514,A,N,A,A,G,BI,E,AV,AG,...,0.592863,0.250877,0.292144,0.358930,0.562570,0.707359,0.632267,0.284889,0.518947,0
2,289595,B,F,D,A,E,BI,A,F,L,...,0.835299,0.837267,0.357749,0.678743,0.575143,0.754098,0.642640,0.958589,0.940756,0
3,290200,B,I,A,A,E,BI,C,K,BM,...,0.369965,0.573459,0.455696,0.800336,0.608690,0.261167,0.493506,0.443034,0.342730,0
4,3547,A,B,D,A,F,BI,A,AS,AX,...,0.694353,0.577407,0.721532,0.487756,0.610030,0.668544,0.360062,0.763265,0.265792,0


In [13]:
df_train.columns

Index(['id', 'cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
       'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15',
       'cat16', 'cat17', 'cat18', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4',
       'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'target'],
      dtype='object')

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
?train_test_split

Signature: train_test_split(*arrays, **options)
Docstring:
Split arrays or matrices into random train and test subsets

Quick utility that wraps input validation and
``next(ShuffleSplit().split(X, y))`` and application to input data
into a single call for splitting (and optionally subsampling) data in a
oneliner.

Read more in the :ref:`User Guide <cross_validation>`.

Parameters
----------
*arrays : sequence of indexables with same length / shape[0]
    Allowed inputs are lists, numpy arrays, scipy-sparse
    matrices or pandas dataframes.

test_size : float or int, default=None
    If float, should be between 0.0 and 1.0 and represent the proportion
    of the dataset to include in the test split. If int, represents the
    absolute number of test samples. If None, the value is set to the
    complement of the train size. If ``train_size`` is also None, it will
    be set to 0.25.

train_size : float or int, default=None
    If float, should be between 0.0 and 1.0 and represent the
 

In [16]:
cat_col = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
       'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15',
       'cat16', 'cat17', 'cat18']
cont_col = ['cont0', 'cont1', 'cont2', 'cont3', 'cont4',
       'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10']

In [17]:
for col in cat_col:
    df_train[col] = df_train[col].apply(hash_cat_two)
    print("done ", col)

done  cat0
done  cat1
done  cat2
done  cat3
done  cat4
done  cat5
done  cat6
done  cat7
done  cat8
done  cat9
done  cat10
done  cat11
done  cat12
done  cat13
done  cat14
done  cat15
done  cat16
done  cat17
done  cat18


In [18]:
X = df_train.drop(["id", "target"], axis=1)
y = df_train["target"]


In [19]:
X.shape

(240000, 30)

In [20]:
y.shape

(240000,)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=111)

In [23]:
import lightgbm as gbm

In [24]:
df_train["cat0"].unique()

array([0, 1])

In [25]:
train_data = gbm.Dataset(X_train, label=y_train, categorical_feature=cat_col)

In [26]:
validation_data = gbm.Dataset(X_test, label=y_test, reference=train_data)

In [27]:
lgbm_params = {
    'boosting': 'gbdt',          # dart (drop out trees) often performs better
    'application': 'binary',     # Binary classification
    'learning_rate': 0.02,       # Learning rate, controls size of a gradient descent step
    'min_data_in_leaf': 20,      # Data set is quite small so reduce this a bit
    'feature_fraction': 0.7,     # Proportion of features in each boost, controls overfitting
    'num_leaves': 100,            # Controls size of tree since LGBM uses leaf wise splits
    'metric': 'auc',  # Area under ROC curve as the evaulation metric
}

In [28]:
num_round = 1000
bst = gbm.train(lgbm_params, train_data, num_round, valid_sets=[validation_data], early_stopping_rounds=10, verbose_eval=100)

/home/cpu11453/miniconda3/envs/nlp/lib/python3.7/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/cpu11453/miniconda3/envs/nlp/lib/python3.7/site-packages/lightgbm/basic.py:1555: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat0', 'cat1', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Info] Number of positive: 57309, number of negative: 158691
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.080287 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3390
[LightGBM] [Info] Number of data points in the train set: 216000, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265319 -> initscore=-1.018501
[LightGBM] [Info] Start training from score -1.018501
Training until validation scores don't improve for 10 rounds


/home/cpu11453/miniconda3/envs/nlp/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/cpu11453/miniconda3/envs/nlp/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[100]	valid_0's auc: 0.887654
[200]	valid_0's auc: 0.891046
[300]	valid_0's auc: 0.893012
[400]	valid_0's auc: 0.893849
Early stopping, best iteration is:
[424]	valid_0's auc: 0.893912


In [29]:
# ?gbm.train

In [30]:
df_test = pd.read_csv(test_path)
df_test.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,150904,A,F,A,A,G,BI,A,AN,BM,...,0.637685,0.301544,0.790893,0.488015,0.614380,0.539349,0.411866,0.661432,0.486744,0
1,168869,A,F,C,A,E,BI,C,AK,AX,...,0.324491,0.349067,0.786160,0.554579,0.486805,0.390730,0.375106,0.569209,0.459002,0
2,125842,A,N,A,A,E,BI,C,E,BM,...,0.279434,0.346821,0.208250,0.618792,0.494801,0.367308,0.387041,0.474525,0.402563,0
3,37203,A,I,A,A,E,BI,A,N,AF,...,0.305931,0.321362,0.596346,0.797107,0.575397,0.326885,0.353614,0.258005,0.357856,0
4,353336,A,M,C,A,E,BI,G,E,BM,...,0.190550,0.578725,0.777659,0.773500,0.703389,0.239250,0.242388,0.283970,0.311369,1


In [36]:
df_test2 = df_test.drop(["id", "target"], axis=1)

In [37]:
for col in cat_col:
    df_test2[col] = df_test2[col].apply(hash_cat_two)
    print("done ", col)

done  cat0
done  cat1
done  cat2
done  cat3
done  cat4
done  cat5
done  cat6
done  cat7
done  cat8
done  cat9
done  cat10
done  cat11
done  cat12
done  cat13
done  cat14
done  cat15
done  cat16
done  cat17
done  cat18


In [38]:
# ?bst

In [39]:
df_test2.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10
0,0,4,0,0,9,21,0,51,70,14,...,0.616445,0.637685,0.301544,0.790893,0.488015,0.614380,0.539349,0.411866,0.661432,0.486744
1,0,4,12,0,14,21,12,79,55,0,...,0.350460,0.324491,0.349067,0.786160,0.554579,0.486805,0.390730,0.375106,0.569209,0.459002
2,0,3,0,0,14,21,12,14,70,6,...,0.323298,0.279434,0.346821,0.208250,0.618792,0.494801,0.367308,0.387041,0.474525,0.402563
3,0,5,0,0,14,21,0,3,47,14,...,0.331975,0.305931,0.321362,0.596346,0.797107,0.575397,0.326885,0.353614,0.258005,0.357856
4,0,11,12,0,14,21,9,14,70,0,...,0.240007,0.190550,0.578725,0.777659,0.773500,0.703389,0.239250,0.242388,0.283970,0.311369


In [40]:
pred = bst.predict(df_test2, num_iteration=bst.best_iteration)

In [41]:
df_pred_final = df_test[["id"]]

In [42]:
df_pred_final["target"] = pred

/home/cpu11453/miniconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [43]:
df_pred_final.head()

,id,target
0,150904,0.055263
1,168869,0.060864
2,125842,0.106717
3,37203,0.152398
4,353336,0.489305


In [ ]:
# df_pred_final.to_csv('submission.csv', index=False)

In [44]:
pred

array([0.05526269, 0.06086388, 0.10671689, ..., 0.08300336, 0.84886748,
       0.02742586])

# evaluation 

In [47]:
from sklearn.metrics import roc_auc_score

In [48]:
y_test_label = df_test["target"]

In [49]:
roc_auc_score(y_test_local, pred)

0.8961625009603029